In [1]:
import yaml
import os
import psycopg2
import redis
from dotenv import load_dotenv
import json
import hashlib
from datetime import datetime

In [2]:
load_dotenv()

True

In [3]:
pg_user = os.getenv('POSTGRES_USER')
pg_password = os.getenv('POSTGRES_PASSWORD')
pg_db = os.getenv('POSTGRES_DB')
pg_host = os.getenv('POSTGRES_HOST', 'localhost')
pg_port = os.getenv('POSTGRES_PORT', '5432')


In [4]:
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_db,
    user=pg_user,
    password=pg_password,
    port=pg_port
)

In [5]:
pg_cursor = pg_conn.cursor()
print("Successfully connected to PostgreSQL.")

Successfully connected to PostgreSQL.


In [6]:
# This is the Data I want to add

portfolio_data = {
    "name": "Portfolio Manager",
    "description": "An AI Blockchain driven portfolio management system that optimizes asset allocation and risk management in Aptos.",
    "tags": ["AI", "Blockchain", "Deep Learning"],
    "creation_date": "March-16-2025 07:15:39",
    "owner": "0x22b7e94bb08eb07d59d1a56345e572a5b4409563bc0c0c8fd3eec0ec0bea8d46",
    "last_edited": "March-23-2025 08:42:29",
    "license": "MIT",
    "fork": "Original",
    "socials": "X: @harshp_16 | GitHub: harshpoddar03",
    "deployment_status": "Active",
    "md5": "e67044f2cc088c8f5c359faf3c21e7e1",
    "version": "v0.3",
    "published_date": "March-23-2025 08:42:49",
    "publish_hash": "0x1c66d49cc66cdc29d45d93b8388acdd62079cf18713de64a84c5260ed40ba0bb",
    "chain": "APTOS Testnet",
    "chain_id": "1",
    "chain_status": "Active",
    "chain_explorer": "explorer.aptoslabs.com/?network=testnet",
    "contract_name": "NeuraSynthesis",
    "contract_version": "v0.01",
    "contract_id": "0x48b3475fd2c5d2ae55b80154ea006e6ed6ffb78c8e7dbfd14288168d7da3f7e6",
    "nft_id": "NFT-001",
    "nft_mint_hash": "0x20dd388a619f40aaabc36da3314278d0ad763ceb814d838e9853cbe944159af3",
    "my_access": "Level 6",
    "no_of_access": "2",
    "monetization": "None"
}

In [7]:
name_owner_concat = portfolio_data["name"] + portfolio_data["owner"]
agent_id = f"PM-{hashlib.md5(name_owner_concat.encode()).hexdigest()}"

In [8]:
agent_id

'PM-3542974ceeaf69a01cd24288f66d52bb'

In [9]:
# check tables in the database
pg_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
tables = pg_cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
contract_details
blockchain_agent_data
nft_access
access_level_table
conversations
messages
flowbuilder_blocks
salt_email
chain_details
user_auth
agent
unpublished_agent
published_agent
metadata


In [10]:
# pg_conn.rollback()

In [11]:
pg_cursor.execute("""
    INSERT INTO "chain_details" (chain_id, chain, chain_status, chain_explorer)
    VALUES (%s, %s, %s, %s)
""", (
    101,
    "SUI",
    "ACTIVE",
    "https://suiscan.xyz/testnet/"
))


In [ ]:
# # 2. Insert into CONTRACT_DETAILS
# pg_cursor.execute("""
#     INSERT INTO "contract_details" ("contract_id", "chain_id", "contract_name", "contract_version")
#     VALUES (%s, %s, %s, %s)
#     ON CONFLICT ("contract_id") DO UPDATE SET
#         "chain_id" = EXCLUDED."chain_id",
#         "contract_name" = EXCLUDED."contract_name",
#         "contract_version" = EXCLUDED."contract_version"
# """, (
#     portfolio_data["contract_id"],
#     portfolio_data["chain_id"],
#     portfolio_data["contract_name"],
#     portfolio_data["contract_version"]
# ))


In [ ]:
# # 3. Ensure user exists in USER_AUTH
# pg_cursor.execute("""
#     INSERT INTO "user_auth" ("user_pub_key","username","email","chain_id","created_at")
#     VALUES (%s, %s, %s, %s, %s)
#     ON CONFLICT ("user_pub_key") DO NOTHING
# """, (
#     portfolio_data["owner"],
#     # give random username and email
#     f"{portfolio_data['owner'][:8]}_user",
#     f"{portfolio_data['owner'][:8]}@example.com",
#     portfolio_data["chain_id"],
#     datetime.now()
# ))


In [ ]:
# # 4. Insert into AGENT
# pg_cursor.execute("""
#     INSERT INTO "agent" (
#         "agent_id", "creation_date", "owner", "status", "tags", "license", "fork", "socials", 
#         "description", "name", "chain_id"
#     )
#     VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#     ON CONFLICT ("agent_id") DO UPDATE SET
#         "status" = EXCLUDED."status",
#         "tags" = EXCLUDED."tags",
#         "license" = EXCLUDED."license",
#         "fork" = EXCLUDED."fork",
#         "socials" = EXCLUDED."socials",
#         "description" = EXCLUDED."description",
#         "name" = EXCLUDED."name"
# """, (
#     agent_id,
#     datetime.strptime(portfolio_data["creation_date"], "%B-%d-%Y %H:%M:%S"),
#     portfolio_data["owner"],
#     portfolio_data["deployment_status"],
#     json.dumps(portfolio_data["tags"]),  # Convert tags list to JSON array
#     portfolio_data["license"],
#     portfolio_data["fork"],
#     json.dumps(portfolio_data["socials"]),
#     portfolio_data["description"],
#     portfolio_data["name"],
#     portfolio_data["chain_id"]
# ))

In [ ]:
# # 5. Insert into PUBLISHED_AGENT
# workflow_json = json.dumps({})

# pg_cursor.execute("""
#     INSERT INTO "published_agent" ("agent_id", "last_edited_time", "workflow", "md5")
#     VALUES (%s, %s, %s, %s)
#     ON CONFLICT ("agent_id") DO UPDATE SET
#         "last_edited_time" = EXCLUDED."last_edited_time",
#         "workflow" = EXCLUDED."workflow",
#         "md5" = EXCLUDED."md5"
# """, (
#     agent_id,
#     datetime.strptime(portfolio_data["last_edited"], "%B-%d-%Y %H:%M:%S"),
#     workflow_json,
#     portfolio_data["md5"]
# ))

In [ ]:
# # 6. Insert into BLOCKCHAIN_AGENT_DATA
# pg_cursor.execute("""
#     INSERT INTO "blockchain_agent_data" (
#         "agent_id", "version", "published_date", "published_hash", 
#         "contract_id", "nft_id", "nft_mint_trx_id"
#     )
#     VALUES (%s, %s, %s, %s, %s, %s, %s)
#     ON CONFLICT ("agent_id") DO UPDATE SET
#         "version" = EXCLUDED."version",
#         "published_date" = EXCLUDED."published_date",
#         "published_hash" = EXCLUDED."published_hash",
#         "contract_id" = EXCLUDED."contract_id",
#         "nft_id" = EXCLUDED."nft_id",
#         "nft_mint_trx_id" = EXCLUDED."nft_mint_trx_id"
# """, (
#     agent_id,
#     portfolio_data["version"],
#     datetime.strptime(portfolio_data["published_date"], "%B-%d-%Y %H:%M:%S"),
#     portfolio_data["publish_hash"],
#     portfolio_data["contract_id"],
#     portfolio_data["nft_id"],
#     portfolio_data["nft_mint_hash"]
# ))


In [ ]:
# # 7. Ensure ACCESS_LEVEL_TABLE entry exists
# access_level = int(portfolio_data["my_access"].split()[-1])

# pg_cursor.execute("""
#     INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
#     VALUES (%s, %s, %s)
#     ON CONFLICT ("access_level") DO NOTHING
# """, (
#     access_level,
#     portfolio_data["my_access"],
#     json.dumps(portfolio_data["description"])
# ))

In [12]:
# 7. Ensure ACCESS_LEVEL_TABLE entry exists
# access_level = int(portfolio_data["my_access"].split()[-1])

# pg_cursor.execute("""
#     INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
#     VALUES (%s, %s, %s)
#     ON CONFLICT ("access_level") DO NOTHING
# """, (
#     5,
#     "Level 5",
#     json.dumps("Basic + Edit + Monetize + share + coowner")
# ))
# make for 6 lvls

pg_cursor.execute("""
    INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
    VALUES (%s, %s, %s)
    ON CONFLICT ("access_level") DO NOTHING
""", (
    1,
    "Level 1",
    json.dumps("Level 1")
))

pg_cursor.execute("""
    INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
    VALUES (%s, %s, %s)
    ON CONFLICT ("access_level") DO NOTHING
""", (
    2,
    "Level 2",
    json.dumps("Level 2")
))

pg_cursor.execute("""
    INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
    VALUES (%s, %s, %s)
    ON CONFLICT ("access_level") DO NOTHING
""", (
    3,
    "Level 3",
    json.dumps("Level 3")
))

pg_cursor.execute("""
    INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
    VALUES (%s, %s, %s)
    ON CONFLICT ("access_level") DO NOTHING
""", (
    4,
    "Level 4",
    json.dumps("Level 4")
))

pg_cursor.execute("""
    INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
    VALUES (%s, %s, %s)
    ON CONFLICT ("access_level") DO NOTHING
""", (
    5,
    "Level 5",
    json.dumps("Level 5")
))

pg_cursor.execute("""
    INSERT INTO "access_level_table" ("access_level", "access_level_name", "descriptions_and_permissions")
    VALUES (%s, %s, %s)
    ON CONFLICT ("access_level") DO NOTHING
""", (
    6,
    "Level 6",
    json.dumps("Level 6")
))





In [ ]:
# # 8. Insert NFT_ACCESS
# pg_cursor.execute("""
#     INSERT INTO "nft_access" ("user_id","nft_id", "access_level", "timestamp")
#     VALUES (%s, %s, %s, %s)
#     ON CONFLICT DO NOTHING
# """, (
#     portfolio_data["owner"],
#     portfolio_data["nft_id"],
#     access_level,
#     datetime.now()
# ))

In [ ]:
# # 9. Insert METADATA
# metadata_json = json.dumps({
#     "documentation": f"# {portfolio_data['name']}\n\n{portfolio_data['description']}"
# })

# pg_cursor.execute("""
#     INSERT INTO "metadata" ("agent_id", "markdown_object")
#     VALUES (%s, %s)
#     ON CONFLICT ("agent_id") DO UPDATE SET
#         "markdown_object" = EXCLUDED."markdown_object"
# """, (
#     agent_id,
#     metadata_json
# ))

In [13]:
# Final commit
pg_conn.commit()
# print(f"✅ Successfully inserted Portfolio Manager with agent_id: {agent_id}")
